You set the trend!
------------------

Today we start letting you look at some twitter data and set the trends. In the last drill, you came up with some ideas and today you can test them out. We'll start slow, you'll work a bit at home and we'll finish next time. Your first algorithm!

**Trends a la Twitter**

First, to set a baseline, let's load up Twitter's trends and remind ourselves about DC. We set up Pandas to display right and then read in the data you downloaded from the Twitter trends drill. 

In [ ]:
from pandas import set_option, read_csv
set_option("display.max_colwidth",140)
set_option("display.max_columns",30)
set_option("display.max_rows",50)

In [ ]:
trends = read_csv('twitter_trending_topics_for_us_120to122_mh2.csv')
trends.shape

That means 700K or so trends in the data set. Let's remind ourselves what the data look like.

In [ ]:
trends.head(5)

So a trend in city, the whole thing ordered by city then position then time we collected the data. Let's start by looking at just Washington, and, following Suman's advice, at only those trends that made it to the top 10. The others, you recall, aren't visible to users -- but are certainly helpful when understanding the behavior of the algorithm.

In [ ]:
trendsdc = trends[(trends["city"]=="Washington") & (trends["position"] <=10)]
trendsdc.shape

To see what's going on, dot charts have been our friend. Maybe they're too close at this point, but it's only really the 4th class so... and we'll see histograms today. Remember, get your own plotly login! Just head to [plotly.com](http://plotly.com) and sign up. My poor account is taking a beating!

In [ ]:
from plotly.plotly import iplot, sign_in
import plotly.graph_objs as go 

sign_in("cocteautt","9psj3t57ti")

In [ ]:
tops = trendsdc["topic_name"].value_counts().index[:50]
trendsdc_tops = trendsdc[trendsdc["topic_name"].isin(tops)]

mydata = [go.Scatter(x=trendsdc_tops["datetime"],y=trendsdc_tops["topic_name"],mode="markers",name="top tweeter")]
mylayout = go.Layout(autosize=False, width=1000,height=1500,margin=go.Margin(l=150,r=50,b=100,t=100,pad=4))
myfigure = go.Figure(data = mydata, layout = mylayout)
iplot(myfigure)

OK we are old hands at that. We found the top 50 trending topics (again, those appearing the the top 10) and can see how they vary over the time we've collected data. We'll come back to this plot and compare what we see from tweets captured in the district.

**A second pass -- A less simple CSV representing 226,988 tweets**

Mike Young worked evern harder and got you a new an improved CSV describing tweets. [Download it from here](http://compute-cuj.org/dc_tweets_0120_to_0123_v2_mh.csv.gz), uncompress it and put it in the same folder as this notebook. 

It contains a 1% sample of all tweets, pulled from the Twitter API. We will see next time that you an pull data from the API too! It's a service that Twitter makes available and is not that hard to use. Well, it is tricky if you want to collect data for a long period of time like Mike has done. But to just get a quick snapshot, it's pretty direct. Fun!

OK, read in the data.

In [ ]:
tweetsdc = read_csv("dc_tweets_0120_to_0123_v2_mh.csv")
tweetsdc.shape

In [ ]:
tweetsdc.head(5)

Your first question might just be, who's out there tweeting? What's the distribution look like (do 5% of the tweeters account for 95% of the tweets, say?). Then, we'll usee good old plotly to tell us when the top 50 tweet.

The desription below tells us that this categorical variable (made up of strings) has 41.5K unique values, meaning separate tweeters are responsible for the 227K tweets. 

In [ ]:
tweetsdc["user's screen_name"].describe()

In [ ]:
tweetsdc["user's screen_name"].value_counts()

The top 5% (about 2000 tweeters) account for just over 40% of all the tweets. OK not sure what we do with that fact, but there's a cottage industry in so-called "power laws" like this. We might be able to infer something about robot action if we cut this down to tweets in a particular topic. The more skewed, the more robot-like? We'll get there.

In [ ]:
tops = tweetsdc["user's screen_name"].value_counts()[:2000]
tops.sum()/226988.0

Now, remake "tops" to consist of just the screen names of the top 50 tweeters. Use plotly to show when they tweet. 

In [ ]:
tops = tweetsdc["user's screen_name"].value_counts()[:50].index

In [ ]:
from plotly.plotly import iplot, sign_in
import plotly.graph_objs as go 

sign_in("cocteautt","9psj3t57ti")

In [ ]:
tops = tweetsdc["user's screen_name"].value_counts().index[:50]
tweetsdc_tops = tweetsdc[tweetsdc["user's screen_name"].isin(tops)]

mydata = [go.Scatter(x=tweetsdc_tops["tweet created_at time"],y=tweetsdc_tops["user's screen_name"],mode="markers",name="top tweeter")]
mylayout = go.Layout(autosize=False, width=1000,height=1000,margin=go.Margin(l=150,r=50,b=100,t=100,pad=4))
myfigure = go.Figure(data = mydata, layout = mylayout)
iplot(myfigure)

What do you see from this plot? Any patterns in time? 

Ultimatey, a dot chart is a great tool, but we've been looking in the articles from Suman at histograms. The good people at ploty have created a nice histogram interface for us to use. Here we look at the distribution of all tweet activity, not just the top 50. How does the histogram and the dot chart above compare?

(And don't mind the warning. It's saying that if you want to make a scatterplot with 200K points, you are crazy. BUT if you want to make a histogram or a boxplot or some other summary, rock on. And so, we rock on...)

In [ ]:
mydata = [go.Histogram(x=tweetsdc["tweet created_at time"])]
mylayout = go.Layout(autosize=False, width=800,height=600,margin=go.Margin(l=150,r=50,b=100,t=100,pad=4))
myfigure = go.Figure(data = mydata, layout = mylayout)
iplot(myfigure)

What does this tell you? 

How do you read a histogram? It's like phrenology, maybe. Bumps. You are after bumps. Each bump is a "cluster," a group of points that are hanging together. It's up to you to find if the reason for the bump is interesting or obvious. The big bumps are obvious (why?). What about the littler ones?

This shows us that as we develop our trending algorithms, we will have very different data totals to work with. It also might be worth looking at trends during the quiet periods. 

Here are the hashtags from the various tweets. NaN means no hashtag.

In [ ]:
tweetsdc["hashtag #1"].value_counts().head(20)

In [ ]:
tweetsdc["hashtag #2"].value_counts().head(20)

In [ ]:
tweetsdc["hashtag #3"].value_counts().head(20)

Here we take the hashtags and "concatenate" them. That is, we create one big data set of time and hashtag. It probably doesn't matter at this point if it's hashtag number 1 or 3. So we create just one column called "hashtag" and one called "tweet created_at time". The Pandas function concat() lets us combine them into one dataframe. 

To do this, we need to make sure the columns of the 3 dataframes have the same names. So we replace their names with "hashtag". This is done by assigning a list of new names to the ".columns" attribute of the dataframe object.

We also sort them by time.

In [ ]:
from pandas import concat

hashtags1 = tweetsdc[["tweet created_at time","hashtag #1"]]
hashtags1.columns = ["tweet created_at time","hashtag"]

hashtags2 = tweetsdc[["tweet created_at time","hashtag #2"]]
hashtags2.columns = ["tweet created_at time","hashtag"]

hashtags3 = tweetsdc[["tweet created_at time","hashtag #3"]]
hashtags3.columns = ["tweet created_at time","hashtag"]

hashtags = concat([hashtags1,hashtags2,hashtags3])

hashtags.shape

In [ ]:
hashtags = hashtags[~hashtags["hashtag"].isnull()]
hashtags = hashtags.sort_values(by="tweet created_at time")
hashtags.shape

OK, now, let's look at some hashtags. How about anything to do with the Women's March. We'll look at the string "womensmarch" and make so-called "case insensitive" matches. We don't care if its womensmarch or WomensMarch or WOMENSMARCH. And using contains() means we are looking for hashtags that contain that string. 

Have a look.

In [ ]:
hashtags[hashtags["hashtag"].str.contains("womensmarch",case=False)].head(10)

And here is the histogram of when this occurred. Again, we can look at a dot-chart but this gives us a better view into the shape of the timing.

In [ ]:
mydata = [go.Histogram(x=hashtags[hashtags["hashtag"].str.contains("womensmarch",case=False)]["tweet created_at time"])]
mylayout = go.Layout(autosize=False, width=800,height=600,margin=go.Margin(l=150,r=50,b=100,t=100,pad=4))
myfigure = go.Figure(data = mydata, layout = mylayout)
iplot(myfigure)

Let's add a couple more. We'll take the hashtags, lowercase them and then use value_counts(). The top 50 show interests other than just the march.

In [ ]:
hashtags["hashtag"].str.lower().value_counts().head(50)

So, let's look at the Patriots and whatever other sports-ish thing I can find in that list. The "|" means "or" again. We'll have more to say about that shortly. But for now, we're looking for "patriots" or "gbvsatl".

In [ ]:
mydata = [go.Histogram(x=hashtags[hashtags["hashtag"].str.contains("womensmarch",case=False)]["tweet created_at time"],name="march",opacity=0.5),
          go.Histogram(x=hashtags[hashtags["hashtag"].str.contains("patriots|gbvsatl",case=False)]["tweet created_at time"],name="patriots",opacity=0.5)]
mylayout = go.Layout(barmode='overlay',autosize=False, width=800,height=600,margin=go.Margin(l=150,r=50,b=100,t=100,pad=4))
myfigure = go.Figure(data = mydata, layout = mylayout)
iplot(myfigure)

Nice! I'm going to toss another in that came from looking at the trending topics at the top of this notebook. "Bellator170". <br><br>
<img src=http://www.mmamadhouse.com/images/2016/10/chael-sonnen-vs-tito-ortiz-bellator-170-640x360-1477145882-800.jpg width=500>
<br><br>
This chap trended, presumably during his fight. We don't have a huge set of references in our data set, but some. 

In [ ]:
hashtags[hashtags["hashtag"]=="Bellator170"].shape

Now, compare this to some of the other lower tweeted hashtags. We'll make first a histogram...

In [ ]:
mydata = [go.Histogram(x=hashtags[hashtags["hashtag"].str.contains("gbvsatl",case=False)]["tweet created_at time"],name="gbvsatl",opacity=0.5),
          go.Histogram(x=hashtags[hashtags["hashtag"].str.contains("patriots",case=False)]["tweet created_at time"],name="patriots",opacity=0.5),
          go.Histogram(x=hashtags[hashtags["hashtag"].str.contains("bellator170",case=False)]["tweet created_at time"],name="bellator",opacity=0.5)]
mylayout = go.Layout(barmode='overlay',autosize=False, width=800,height=600,margin=go.Margin(l=150,r=50,b=100,t=100,pad=4))
myfigure = go.Figure(data = mydata, layout = mylayout)
iplot(myfigure)

... and then a dotchart comparing tweeting activity to trending times. What do you see?

In [ ]:
mytrendsdc = trendsdc[trendsdc["topic_name"].str.contains("bellator170",case=False)]
mytweetsdc = hashtags[hashtags["hashtag"].str.contains("bellator170",case=False)]
                          
mydata = [go.Scatter(x=mytrendsdc["datetime"],y=mytrendsdc["topic_name"],mode="markers",name="trending"),
          go.Scatter(x=mytweetsdc["tweet created_at time"],y=mytweetsdc["hashtag"],mode="markers",name="tweeting")
          ]
mylayout = go.Layout(autosize=False, width=1000,height=500,margin=go.Margin(l=150,r=50,b=100,t=100,pad=4))
myfigure = go.Figure(data = mydata, layout = mylayout)
iplot(myfigure)

Finally, what were people saying, and who was saying it? 

In [ ]:
bell = tweetsdc[~tweetsdc["hashtag #1"].isnull() & tweetsdc["hashtag #1"].str.contains("bellator",case=False)]
bell

In [ ]:
bell["user's screen_name"].value_counts()

Now some works with time...

In [ ]:
tweetsdc.head(5)

In [ ]:
timefocus = tweetsdc[tweetsdc["tweet created_at time"].str.startswith("2017-01-22T01:")]
timefocus.shape

In [ ]:
for t in timefocus["tweet text"]:
    print t

In [ ]:
allwords = []

for t in timefocus["tweet text"]:
    allwords = allwords+t.lower().split(" ")

In [ ]:
from pandas import DataFrame 

allwords = DataFrame(allwords,columns=["word"])
allwords.head(5)

In [ ]:
allwords["word"].value_counts()[0:50]

In [ ]:
allwords["word"].value_counts()[50:100]

In [ ]:
allwords["word"].value_counts()[100:150]

In [ ]:
trendsdc[trendsdc["datetime"].str.startswith("2017-01-22T01:")]

In [ ]:
trendsdc.head(5)